In [1]:
import numpy as np
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import os
import sklearn as sk
from pandas_profiling import ProfileReport
import pickle
from sklearn.linear_model import LinearRegression
import logging as lg

In [2]:
pwd

'C:\\Users\\sivag\\Downloads'

In [11]:
os.chdir('Downloads\\')

In [3]:
lg.basicConfig(filename='linear.log',level=lg.DEBUG, format=' %(asctime)s -%(levelname)s -  %(message)s')

In [4]:
def data_preprocess():
    try:
        
        df = load_dataset()
        lg.info("Starting the data pre processing")
    
        #creating profile
        #lg.info("Starting the data profile")
        #profile = ProfileReport(df, explorative=True)
        #profile.to_file("AI4I_2020.html")
        #lg.info("data profile creation completed with name AI4I_2020.html and saved in ",os.getcwd())
    
        
        # Dropping Torque as Rotational is related (Multi collinear) to it within feature
        df.drop('Torque [Nm]', axis =1, inplace=True)
        
        # Dropping UDI as it is primary key
        df.drop('UDI', axis =1, inplace=True)
        
        #As Type column came from Product ID (first letter), we can drop Product ID
        df.drop('Product ID', axis =1, inplace=True)
        
        #Converting ordinal catergory (L, M, H) in Type to integers
        df['Type'] = df['Type'].str.replace('L','1')
        df['Type'] = df['Type'].str.replace('M','2')
        df['Type'] = df['Type'].str.replace('H','3')
        
        #Changing column names for better appearance
        df.columns = ['Type','Air_temp','Process_temp','Rotaional_speed','Tool_wear','Machine_failure','TWF','HDF','PWF','OSF','RNF']
        
        #Replacing zeros values of df['Tool_wear'] to median as it is not normally distributed.
        df['Tool_wear'].median()
        df['Tool_wear']=df['Tool_wear'].replace({0:'108'})
        
        #Converting object columns to int
        df['Tool_wear'] = df['Tool_wear'].astype(int)
        df['Type'] = df['Type'].astype(int)
    
        return df
    
    except Exception as e:
        lg.exception(str(e))
        lg.info("pre process step failed")
        print(str(e))
        
    else:
        lg.info("pre process step completed successfully")
        
    

In [15]:
def model_predict(linear):
    lg.info("Predicting Air temp from user entry")
    ##ip=input('enter Type(L=1,M=2,H=3),Process temperature [K],Rotational speed [rpm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF')
    #lg.info("user entered input is ",ip)
    try:
        
        lg.info("predicting the air temp...")
        print("predicted air temp is...", linear.predict([[1,308.5,16000,5,0,0,0,0,0,0]]))
        lg.info("sample entry predicted")
    except Exception as e:
        lg.exception(str(e))
        lg.info("predicting step failed")
        print(str(e))

In [6]:
def load_dataset():
    lg.info("Starting the dataset loading")
    try:
        df =pd.read_csv('ai4i2020.csv')
        lg.info("dataset loaded successfully")
        return df
    except Exception as e:
        lg.exception(str(e))
        lg.info("dataset laod not success")
        print(str(e))
    else:
        lg.info("dataset loaded successfully")
    

In [7]:
def save_model(linear):
    
    try:
        #Saving the model
        lg.info('saving the linear reg model')
        file = 'linear_reg_classwork.sav'
        pickle.dump(linear,open(file,'wb'))
        
    except Exception as e:
        lg.info('model couldnot be saved')
        lg.exception(str(e))
        print(str(e))
        
    else:
        lg.info('model saved')


In [8]:
def get_param(linear):
    lg.info('printing params of linear reg model')
    print("Intercept of our model is ",linear.intercept_)
    print("Coefficients of model are",linear.coef_)

In [16]:
class linear_reg:
    def __int__(self, x, y):
        self.x=x
        self.y=y
    def __str__(self):
        print('class to perform Linear Regression with x and y')
        
    try:
        df = data_preprocess()
        
        #Creating dependent and independent variables
        
        x=df[['Type','Process_temp','Rotaional_speed','Tool_wear','Machine_failure','TWF','HDF','PWF','OSF','RNF']]
        #x = df.drop("Air_temp")
        y=df['Air_temp']
        
        
        
    
        ##Creating linear regression object
        linear = LinearRegression()
        lg.info("Model Object created successfully")
        
        # Fitting linear reg obj with our data set
        linear.fit(x,y)
        lg.info("Model fit done successfully")
        
        # Predicting Air_temp with new entry from our fitted model
        lg.info("Predicting sample entry")
        model_predict(linear)
        lg.info("sample entry predicted")
        
        #print the coeffiecients
        get_param(linear)
        
        #Save the model
        save_model(linear)
        
        
    except Exception as e:
        lg.info('could not create linear object model')
        lg.exception(str(e))
        print(str(e))
        
    else:
        lg.info("linear regression model created")
    

predicted air temp is... [300.7675564]
Intercept of our model is  -64.13692389475563
Coefficients of model are [-1.06797859e-02  1.17370880e+00  1.76594113e-04  9.78794212e-05
 -9.08803660e-02  2.02252405e-01  1.78005074e+00  1.47582884e-01
 -4.31951779e-02 -5.67264774e-02]
